In [383]:
import re
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import numpy as np
from sklearn.naive_bayes import *
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

In [384]:
tokenizer = RegexpTokenizer('\w+')
ps = PorterStemmer()
stopwords = []

with open('All_Languages.txt' ,'r+',encoding='utf-8') as f:
    l = f.readlines()
new_l  =[]
for i in l:
    new_l.append(i[:-1])
l = new_l

with open("stopwords.txt", 'r') as f:
    stopwords = f.read()
stopwords = stopwords.split("\n")

sw = set(stopwords)

def getStemReview(reviews):
    review = reviews.lower()
    
    tokens = tokenizer.tokenize(review)
    new_tokens = [w for w in tokens if w not in sw]
    stemmed_token = [ps.stem(token) for token in new_tokens if token not in l]
    
    cleaned_review = ' '.join(stemmed_token)
    return cleaned_review


# Word And Sentence Tokenmization & Stop Removal 
def SentenceTokenization(text):
    sentences = re.compile('[.!?] ').split(text)
    return sentences

def WordTokenization(sentence):
    tokens = re.findall("[\w']+", sentence)
    return tokens

def StopWordRemoval(allwords):
    stopwords = []
    with open("stopwords.txt", 'r') as f:
        stopwords = f.read()
    stopwords = stopwords.split("\n")
    
    usefull_words = [word for word in allwords if word not in stopwords]
    return usefull_words

# Stemming And Lemmatizing
def StemmingAndLemmatizing(usefull_words):
    ps = PorterStemmer()
    
    with open("All_Languages.txt","r+",encoding='utf-8') as f:
        languages = f.read()
        
    keyword_extract = []
    for i in range(len(usefull_words)):
        for j in range(len(usefull_words[i])):
            if usefull_words[i][j].lower() not in languages:
                x = ps.stem(usefull_words[i][j])
                usefull_words[i][j] = x
            else:
                keyword_extract.append(usefull_words[i][j])
    return usefull_words,keyword_extract

def nlp(text):
    sentences = SentenceTokenization(text)

    words = [] 
    for sentence in sentences:
        words_in_a_sent = WordTokenization(sentence)
        words.append(words_in_a_sent)

    usefull_words = []
    for words_in_a_sentence in words:
        usefull_words_in_sent = StopWordRemoval(words_in_a_sentence)
        usefull_words.append(usefull_words_in_sent)

    usefull_words,keywords = StemmingAndLemmatizing(usefull_words)
    return usefull_words,keywords

In [385]:
training_data = pd.read_csv("DataSet3.csv")

In [386]:
training_data

,X,y
0,3-frcr part 2b preparation course (latest edit...,2b
1,cissp practice tests - all domains - 2020 - 42...,420
2,rhythm #1: intro - turn 8th note to 16th note ...,8th
3,sap abap programming for beginners - online tr...,abap
4,debugging sap abap code for non programmers,abap
...,...,...
3885,introduction to μλ,μλ
3886,basics of μλ,μλ
3887,getting started with μλ,μλ
3888,roadmap for μλ,μλ


In [387]:
training_data = training_data.dropna()

In [388]:
X_train,y_train  = training_data.iloc[:,0].values,training_data.iloc[:,1].values
# X_train = ["Introduction to Android","Introduction to angularjs","Basics of Python"]
# y_train = ["android","angularjs","python"]

In [389]:
for i in range(len(X_train)):
    X_train[i],keyword = nlp(X_train[i].lower())
    X_train[i] = " ".join(X_train[i][0])

In [390]:
cv = CountVectorizer(ngram_range=(1,1))
x_vec = cv.fit_transform(X_train).toarray()
x_vec

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int64)

In [391]:
d = dict()
rd= dict()
for i in range(len(l)):
    d[l[i]]=i
    rd[i]= l[i]
    
y_train_num = []
for i in range(len(y_train)):
    y_train_num.append(d[y_train[i]])

In [392]:
model = MultinomialNB()
model.fit(x_vec,y_train_num)

MultinomialNB()

In [423]:
x_test = ["roadmap to js ","i like android","Basics of Python","roadmap of javascript","Getting Started with js","I like kotlin not android"]
for i in range(len(x_test)):
    x_test[i],keword = nlp(x_test[i].lower())
    x_test[i] = " ".join(x_test[i][0])
x_text_vec = cv.transform(x_test).toarray()

In [424]:
y_pred=model.predict(x_text_vec)
print(y_pred)

[108  38 485 326 108 342]


In [425]:
for i in y_pred:
    print(rd[i])

batch
android
python
javascript
batch
kotlin
